In [ ]:
!pip install -U sentence-transformers openai

In [31]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from sentence_transformers import SentenceTransformer
import pandas as pd

In [3]:
model = SentenceTransformer('BAAI/bge-m3')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [13]:
def get_embedding(text):
    return list(model.encode(text))

In [ ]:
embedding_result = get_embedding('저는 배가 고파요')
print(embedding_result)

In [18]:
data = ['저는 배가 고파요',
        '저기 배가 지나가네요',
        '굶어서 허기가 지네요',
        '허기 워기라는 게임이 있는데 즐거워',
        '스팀에서 재밌는 거 해야지',
        '스팀에어프라이어로 연어구이 해먹을거야']

df = pd.DataFrame(data, columns=['text'])
df

,text
0,저는 배가 고파요
1,저기 배가 지나가네요
2,굶어서 허기가 지네요
3,허기 워기라는 게임이 있는데 즐거워
4,스팀에서 재밌는 거 해야지
5,스팀에어프라이어로 연어구이 해먹을거야


In [19]:
df['embedding'] = df.apply(lambda row: get_embedding(
        row.text
    ), axis=1)

In [20]:
df

,text,embedding
0,저는 배가 고파요,"[0.02162664, 0.009280825, -0.052108232, -0.019..."
1,저기 배가 지나가네요,"[-0.003345435, 0.008186257, -0.07204163, -0.00..."
2,굶어서 허기가 지네요,"[0.0004956656, 0.027304389, -0.060060907, -0.0..."
3,허기 워기라는 게임이 있는데 즐거워,"[-0.012998753, 0.01903905, -0.07117989, -0.004..."
4,스팀에서 재밌는 거 해야지,"[-0.03540189, 0.0022580686, -0.049874943, -0.0..."
5,스팀에어프라이어로 연어구이 해먹을거야,"[-0.028986646, 0.0040245336, -0.032376718, -0...."


In [28]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

def return_answer_candidate(df, query):
    # query라고 하는 텍스트가 들어오면 get_embedding이라는 함수를 통해서 벡터값을 얻음.
    # query라고 하는 텍스트의 임베딩 값은 query_embedding에 저장이 됩니다.
    query_embedding = get_embedding(
        query
    )

    # query라는 텍스트가 임베딩이 된 query_embedding과
    # 데이터프레임 df의 embedding 열에 있는 모든 임베딩 벡터값들과 유사도를 계산을 하여
    # similarity 열에다가 각각의 유사도 점수를 기록.
    df["similarity"] = df.embedding.apply(lambda x: cos_sim(np.array(x),
                                                            np.array(query_embedding)))

    # similarity 열에 있는 유사도 값 기준으로 상위 3개의 행만 반환
    results_co = df.sort_values("similarity",
                                ascending=False,
                                ignore_index=True)
    return results_co.head(3)

In [29]:
sim_result = return_answer_candidate(df, '아무 것도 안 먹었더니 꼬르륵 소리가나네')
sim_result

,text,embedding,similarity
0,굶어서 허기가 지네요,"[0.0004956656, 0.027304389, -0.060060907, -0.0...",0.566034
1,저는 배가 고파요,"[0.02162664, 0.009280825, -0.052108232, -0.019...",0.490944
2,허기 워기라는 게임이 있는데 즐거워,"[-0.012998753, 0.01903905, -0.07117989, -0.004...",0.477863
